In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
from IPython.display import display
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv, split_train_test_data

2023-06-17 13:55:02.756940: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-17 13:55:02.835676: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 13:55:03.189998: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 13:55:03.192101: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 13:55:04.339355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
from sklearn.preprocessing import MinMaxScaler
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
scaler = MinMaxScaler()
print('type=', type(data))
a = scaler.fit(data)
print(scaler.fit(data))
print(scaler.data_max_)
print('a', a)
# NFEATS= 3
# scalers = [MinMaxScaler(feature_range=(-1,1)) for i in range(NFEATS)] 
# scalers

type= <class 'list'>
MinMaxScaler()
[ 1. 18.]
a MinMaxScaler()


In [10]:
#mio
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from IPython.display import display
import sys
sys.path.append('/code/src/')
from my_functions import train_test_time_series, evaluate_forecast, load_dataframe_from_csv, split_train_test_data
def create_sequences(data, sequence_length, output_length):
    X, Y = [], []
    shape = data.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        rows, cols = data.shape[0], 1
        data = data.reshape(rows,cols)
    else:
        rows, cols = data.shape
    # Generar los arreglo
    for i in range(rows- sequence_length):
        X.append(data[i: i + sequence_length, 0: cols])
        Y.append(data[i + sequence_length : i + sequence_length + output_length, -1].reshape(output_length,1))
    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)
    return X, Y
def train_test_split(data):
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]
    return train_data, test_data
def escalar_dataset(data_input):
    NFEATS = data_input['x_tr'].shape[2]

    # Generar listado con "scalers"
    # scalers = [MinMaxScaler(feature_range=(0,1)) for i in range(NFEATS)] #feature_rangetuple (min, max), default=(0, 1)
    scalers = [StandardScaler() for i in range(NFEATS)] #feature_rangetuple (min, max), default=(0, 1)

    # Arreglos que contendrán los datasets escalados
    x_tr_s = np.zeros(data_input['x_tr'].shape)
    # x_vl_s = np.zeros(data_input['x_vl'].shape)
    x_ts_s = np.zeros(data_input['x_ts'].shape)
    y_tr_s = np.zeros(data_input['y_tr'].shape)
    # y_vl_s = np.zeros(data_input['y_vl'].shape)
    y_ts_s = np.zeros(data_input['y_ts'].shape)

    # Escalamiento: se usarán los min/max del set de entrenamiento para
    # escalar la totalidad de los datasets
    
    # Escalamiento Xs
    for i in range(NFEATS):
        x_tr_s[:,:,i] = scalers[i].fit_transform(data_input['x_tr'][:,:,i])
        # x_vl_s[:,:,i] = scalers[i].transform(x_vl[:,:,i])
        x_ts_s[:,:,i] = scalers[i].transform(data_input['x_ts'][:,:,i])
    
    # Escalamiento Ys
    y_tr_s[:,:,0] = scalers[-1].fit_transform(data_input['y_tr'][:,:,0])
    # y_vl_s[:,:,0] = scalers[-1].transform(y_vl[:,:,0])
    y_ts_s[:,:,0] = scalers[-1].transform(data_input['y_ts'][:,:,0])

    # Conformar ` de salida
    data_scaled = {
        'x_tr_s': x_tr_s, 'y_tr_s': y_tr_s,
        # 'x_vl_s': x_vl_s, 'y_vl_s': y_vl_s,
        'x_ts_s': x_ts_s, 'y_ts_s': y_ts_s,
    }

    return data_scaled, scalers[0]

def main():
    path_load = "../../data/"
    file_name = "processed/df_time_monthly.csv"
    full_path = path_load + file_name
    data = load_dataframe_from_csv(full_path)
    data.index.freq = 'M'
    #
    INPUT_LENGTH = 3
    OUTPUT_LENGTH = 1
    train_data, test_data =train_test_split(data['cost'])
    # Imprimir en pantalla el tamaño de cada subset
    print(f'Tamaño set de entrenamiento: {train_data.shape}')
    print(f'Tamaño set de prueba: {test_data.shape}')

    x_tr, y_tr = create_sequences(train_data.values, INPUT_LENGTH, OUTPUT_LENGTH)
    x_ts, y_ts = create_sequences(test_data.values, INPUT_LENGTH, OUTPUT_LENGTH)
    print('Tamaños entrada (BATCHES x INPUT_LENGTH x FEATURES) y de salida (BATCHES x OUTPUT_LENGTH x FEATURES)')
    print(f'Set de entrenamiento - x_tr: {x_tr.shape}, y_tr: {y_tr.shape}')
    print(f'Set de prueba - x_ts: {x_ts.shape}, y_ts: {y_ts.shape}')

    # Preprocesamiento de los datos
    data_in = {
    'x_tr': x_tr, 'y_tr': y_tr,
    # 'x_vl': x_vl, 'y_vl': y_vl,
    'x_ts': x_ts, 'y_ts': y_ts,
    }
    data_s, scaler = escalar_dataset(data_in)
    # Extraer subsets escalados
    x_tr_s, y_tr_s = data_s['x_tr_s'], data_s['y_tr_s']
    x_ts_s, y_ts_s = data_s['x_ts_s'], data_s['y_ts_s']
    
    # Verificación
    print(f'Min x_tr/x_ts sin escalamiento: {x_tr.min()}/{x_ts.min()}')
    print(f'Min x_tr/x_ts con escalamiento: {x_tr_s.min()}/{x_ts_s.min()}')
    
    print(f'\nMin y_tr/y_ts sin escalamiento: {y_tr.min()}/{y_ts.min()}')
    print(f'Min y_tr/y_ts con escalamiento: {y_tr_s.min()}/{y_ts_s.min()}')
    
    print(f'\nMax x_tr/x_ts sin escalamiento: {x_tr.max()}/{x_ts.max()}')
    print(f'Max x_tr/x_ts con escalamiento: {x_tr_s.max()}/{x_ts_s.max()}')
    
    print(f'\nMax y_tr/y_ts sin escalamiento: {y_tr.max()}/{y_ts.max()}')
    print(f'Max y_tr/y_ts con escalamiento: {y_tr_s.max()}/{y_ts_s.max()}')

    # Ajustar parámetros para reproducibilidad del entrenamiento
    tf.random.set_seed(123)
    tf.config.experimental.enable_op_determinism()
    
    # El modelo
    N_UNITS = 64 # Tamaño del estado oculto (h) y de la celda de memoria (c)
    INPUT_SHAPE = (x_tr_s.shape[1], x_tr_s.shape[2]) # 3 (meses) x 1 (feature)
    
    modelo = Sequential()
    modelo.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE))
    modelo.add(Dense(OUTPUT_LENGTH, activation='linear'))

    # Compilación
    optimizador = RMSprop(learning_rate=5e-5)
    modelo.compile(
        optimizer = optimizador,
        loss="mean_squared_error",
    )
    
    # Entrenamiento (aproximadamente 1 min usando GPU)
    EPOCHS = 80 # Hiperparámetro
    BATCH_SIZE = 256 # Hiperparámetro
    historia = modelo.fit(
        x = x_tr_s,
        y = y_tr_s,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        # validation_data = (x_vl_s, y_vl_s),
        verbose=2
    )
    rmse_tr = modelo.evaluate(x=x_tr_s, y=y_tr_s, verbose=0)
    # rmse_vl = modelo.evaluate(x=x_vl_s, y=y_vl_s, verbose=0)
    rmse_ts = modelo.evaluate(x=x_ts_s, y=y_ts_s, verbose=0)

    # Imprimir resultados en pantalla
    print('Comparativo desempeños:')
    print(f'  RMSE train:\t {rmse_tr:.3f}')
    # print(f'  RMSE val:\t {rmse_vl:.3f}')
    print(f'  RMSE test:\t {rmse_ts:.3f}')

    
if __name__ == "__main__":
    main()


Tamaño set de entrenamiento: (35,)
Tamaño set de prueba: (9,)
Tamaños entrada (BATCHES x INPUT_LENGTH x FEATURES) y de salida (BATCHES x OUTPUT_LENGTH x FEATURES)
Set de entrenamiento - x_tr: (32, 3, 1), y_tr: (32, 1, 1)
Set de prueba - x_ts: (6, 3, 1), y_ts: (6, 1, 1)
Min x_tr/x_ts sin escalamiento: 645200/2505194
Min x_tr/x_ts con escalamiento: -1.3074767890295889/0.44819263096457684

Min y_tr/y_ts sin escalamiento: 645200/2505194
Min y_tr/y_ts con escalamiento: -1.305147127618661/0.37938306465445154

Max x_tr/x_ts sin escalamiento: 4345208/5296946
Max x_tr/x_ts con escalamiento: 2.330258246350595/3.556685010756135

Max y_tr/y_ts sin escalamiento: 4345208/4466767
Max y_tr/y_ts con escalamiento: 2.0458180839100075/2.155909732400555
Epoch 1/80


2023-06-17 15:21:16.073136: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-17 15:21:16.074265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-17 15:21:16.075093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 - 1s - loss: 1.0429 - 1s/epoch - 1s/step
Epoch 2/80
1/1 - 0s - loss: 1.0395 - 4ms/epoch - 4ms/step
Epoch 3/80
1/1 - 0s - loss: 1.0370 - 6ms/epoch - 6ms/step
Epoch 4/80
1/1 - 0s - loss: 1.0350 - 7ms/epoch - 7ms/step
Epoch 5/80
1/1 - 0s - loss: 1.0331 - 4ms/epoch - 4ms/step
Epoch 6/80
1/1 - 0s - loss: 1.0314 - 5ms/epoch - 5ms/step
Epoch 7/80
1/1 - 0s - loss: 1.0298 - 6ms/epoch - 6ms/step
Epoch 8/80
1/1 - 0s - loss: 1.0283 - 14ms/epoch - 14ms/step
Epoch 9/80
1/1 - 0s - loss: 1.0269 - 8ms/epoch - 8ms/step
Epoch 10/80
1/1 - 0s - loss: 1.0255 - 4ms/epoch - 4ms/step
Epoch 11/80
1/1 - 0s - loss: 1.0242 - 5ms/epoch - 5ms/step
Epoch 12/80
1/1 - 0s - loss: 1.0229 - 6ms/epoch - 6ms/step
Epoch 13/80
1/1 - 0s - loss: 1.0216 - 5ms/epoch - 5ms/step
Epoch 14/80
1/1 - 0s - loss: 1.0204 - 5ms/epoch - 5ms/step
Epoch 15/80
1/1 - 0s - loss: 1.0192 - 5ms/epoch - 5ms/step
Epoch 16/80
1/1 - 0s - loss: 1.0180 - 5ms/epoch - 5ms/step
Epoch 17/80
1/1 - 0s - loss: 1.0168 - 6ms/epoch - 6ms/step
Epoch 18/80
1/1 -

2023-06-17 15:21:18.188192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [32,3,1]
	 [[{{node Placeholder/_10}}]]
2023-06-17 15:21:18.188445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [32,1,1]
	 [[{{node Placeholder/_11}}]]
2023-06-17 15:21:18.376883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split

Comparativo desempeños:
  RMSE train:	 0.950
  RMSE test:	 1.122


In [5]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

# def main():
#     path_load = "../../data/"
#     file_name = "processed/df_time_monthly.csv"
#     full_path = path_load + file_name
#     data = load_dataframe_from_csv(full_path)
#     data.index.freq = 'M'
#     print(data.head(5))
    
#     train_size = int(len(data) * 0.8)
#     train_data = data[:train_size]
#     test_data = data[train_size:]
#     print('--- train_data: ', train_data.shape)
#     print('--- test_data: ', test_data.shape)
    
#     # Preprocesamiento de los datos
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_data[["cost"]])
    
#     # Crear secuencias de entrenamiento
#     sequence_length = 3  # Longitud de la secuencia
#     X_train, y_train = create_sequences(train_scaled, sequence_length)
    
#     # Crear y entrenar el modelo LSTM
#     model = create_lstm_model(sequence_length)
#     model.fit(X_train, y_train, epochs=100, verbose=0)
    
#     # Preparar los datos de prueba
#     test_scaled = scaler.transform(test_data[["cost"]])
#     X_test, y_test = create_sequences(test_scaled, sequence_length)
    
#     # Realizar predicciones
#     predictions = model.predict(X_test)
    
#     # Invertir la escala de las predicciones
#     predictions = scaler.inverse_transform(predictions)
    
#     # Crear un DataFrame con las predicciones y las fechas correspondientes
#     prediction_dates = test_data['date'][sequence_length:]

#     predictions_df = pd.DataFrame(predictions, index=prediction_dates, columns=["predicted_cost"])
#     display(predictions_df)

#     # predictions = []
#     # print('len(X_test)', len(X_test))
#     # for i in range(len(X_test)):
#     #     prediction = model.predict(np.array([X_test[i]]))
#     #     predictions = scaler.inverse_transform(predictions)
#     #     predictions.append(prediction[0])
    
#     # prediction_dates = test_data['date'][sequence_length:]
#     # predictions_df = pd.DataFrame(predictions, index=prediction_dates, columns=["predicted_cost"])
#     # display(predictions_df)

# def load_dataframe_from_csv(file_path):
#     return pd.read_csv(file_path)

# def create_sequences(data, sequence_length):
#     X = []
#     y = []
#     for i in range(len(data) - sequence_length):
#         X.append(data[i:i+sequence_length])
#         y.append(data[i+sequence_length])
#     return np.array(X), np.array(y)

# def create_lstm_model(sequence_length):
#     model = Sequential()
#     model.add(LSTM(64, input_shape=(sequence_length, 1)))
#     model.add(Dense(1))
#     model.compile(loss="mean_squared_error", optimizer="adam")
#     return model

# if __name__ == "__main__":
#     main()


In [77]:
# def create_sequences(data, sequence_length):
#     X = []
#     y = []
#     for i in range(len(data) - sequence_length):
#         X.append(data.iloc[i:i+sequence_length].values)
#         y.append(data.iloc[i+sequence_length]["cost"])
#     return np.array(X), np.array(y)

# def main():
#     path_load = "../../data/"
#     file_name = "processed/df_time_monthly.csv"
#     full_path = path_load + file_name
#     data = load_dataframe_from_csv(full_path)
#     data.index.freq = 'M'
#     display(data.head(5))
#     print(type(data))
    
#     train_size = int(len(data) * 0.8)
#     train_data = data[:train_size]
#     test_data = data[train_size:]
#     print('--- train_data: ', train_data.shape)
#     print('--- test_data: ', test_data.shape)
#     raise SystemExit
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_data)
#     test_scaled = scaler.transform(test_data)
    
#     train_scaled = pd.DataFrame(train_scaled, columns=data.columns)
#     test_scaled = pd.DataFrame(test_scaled, columns=data.columns)

#     sequence_length = 5
#     X_train, y_train = create_sequences(train_scaled, sequence_length)
#     X_test, y_test = create_sequences(test_scaled, sequence_length)
#     print(X_train.shape)
#     print(y_train.shape)

#     model = Sequential()
#     model.add(LSTM(64, activation='relu', input_shape=(sequence_length, train_scaled.shape[1])))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse')

#     model.fit(X_train, y_train, epochs=10, batch_size=16)

#     train_loss = model.evaluate(X_train, y_train, verbose=0)
#     test_loss = model.evaluate(X_test, y_test, verbose=0)
#     print(f'Train Loss: {train_loss:.4f}')
#     print(f'Test Loss: {test_loss:.4f}')

#     # Paso 8: Realizar predicciones
#     predictions = model.predict(X_test)
#     predictions = np.reshape(predictions, (-1, 1))  # Ajustar la forma de las predicciones
#     predictions = scaler.inverse_transform(predictions)  # Invertir la escala utilizando el mismo escalador
    
#     # Crear un DataFrame con las predicciones
#     predictions_df = pd.DataFrame(predictions, columns=["predicted_cost"])
#     predictions_df.index = test_data.index[sequence_length:]  # Ajustar el índice para que coincida con las fechas correspondientes
    
#     # Imprimir las predicciones
#     print(predictions_df)



# if __name__ == "__main__":
#     main()


In [63]:
# def create_sequences(data, sequence_length):
#     X = []
#     y = []
#     for i in range(len(data) - sequence_length):
#         X.append(data.iloc[i:i+sequence_length].values)
#         y.append(data.iloc[i+sequence_length]["cost"])
#     return np.array(X), np.array(y)

# def main():
#     path_load = "../../data/"
#     file_name = "processed/df_time_monthly.csv"
#     full_path = path_load + file_name
#     data = load_dataframe_from_csv(full_path)
#     data.index.freq = 'M'
#     # train_X, train_y, test_X, test_y = split_train_test_data(data, 'cost')
    
#     # Paso 2: Dividir los datos en conjuntos de entrenamiento y prueba
#     train_size = int(len(data) * 0.8)  # El 80% de los datos se utiliza para entrenamiento
#     train_data = data[:train_size]
#     test_data = data[train_size:]
#     print('--- train_data: ', train_data.shape)
#     print('--- test_data: ', test_data.shape)
#     # Paso 3: Preprocesamiento de datos
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_data)
#     test_scaled = scaler.transform(test_data)
#     # print('---type test_data: ', type(test_data), test_data.shape)
#     # print('---type test_scaled: ', type(test_scaled), test_scaled.shape)
#     # display(test_data.head(2))
#     # display(test_scaled[:2])
#     # Convertir las matrices NumPy en DataFrames
#     train_scaled = pd.DataFrame(train_scaled, columns=data.columns)
#     test_scaled = pd.DataFrame(test_scaled, columns=data.columns)

#     # Paso 4: Crear secuencias de entrada y salida
#     sequence_length = 5  # Longitud de la secuencia de entrada
#     X_train, y_train = create_sequences(train_scaled, sequence_length)
#     X_test, y_test = create_sequences(test_scaled, sequence_length)
#     print(X_train.shape)
#     print(y_train.shape)

#     # Paso 5: Construir el modelo LSTM
#     model = Sequential()
#     model.add(LSTM(64, activation='relu', input_shape=(sequence_length, train_scaled.shape[1])))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse')
#     # Paso 6: Entrenar el modelo
#     model.fit(X_train, y_train, epochs=10, batch_size=16)
#     # Paso 7: Evaluar el modelo
#     train_loss = model.evaluate(X_train, y_train, verbose=0)
#     test_loss = model.evaluate(X_test, y_test, verbose=0)
#     print(f'Train Loss: {train_loss:.4f}')
#     print(f'Test Loss: {test_loss:.4f}')
    
#     # Paso 8: Realizar predicciones
#     # Paso 8: Realizar predicciones
#     predictions = model.predict(X_test)
#     scaler_predictions = MinMaxScaler()
#     scaler_predictions.fit(data)  # Ajustar el escalador a los datos originales sin escalar
#     predictions = np.reshape(predictions, (-1, train_scaled.shape[1]))  # Ajustar la forma de las predicciones
#     predictions = scaler_predictions.inverse_transform(predictions)  # Invertir la escala
    
#     print('---train_scaled', train_scaled.shape)
#     print('-----X_test', X_test.shape)
#     print('---predictions.shape', predictions.shape)
#     print('predictions:', predictions)


# if __name__ == "__main__":
#     main()

### Creando un modelo de datos

In [6]:
model = models.Sequential()
model.add(layers.Dense(512,activation='relu', input_shape=(28*28,)))
model.add(layers.Dense(10,activation='softmax'))

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Limpieza de datos

In [8]:
x_train = train_data.reshape((60000,28*28))
x_train = x_train.astype('float32')/255

x_test = test_data.reshape((10000,28*28))
x_test = x_test.astype('float32')/255
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)
train_labels[0]

5

## Entrenando el modelo

In [9]:
model.fit(x_train, y_train, epochs=5, batch_size=128) # demora

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2611 - accuracy: 0.9247
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1072 - accuracy: 0.9686
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0707 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0512 - accuracy: 0.9844
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0378 - accuracy: 0.9887


## Evaluando sobre data de test

In [10]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.0700 - accuracy: 0.9789


[0.07004410028457642, 0.9789000153541565]